In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
from keras.models import Model, load_model
#from keras.applications import ResNet50
from keras.optimizers import Adam
from keras.layers import Flatten, Dense, Dropout, Input
from keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import math
from keras import backend as K  
from keras.models import load_model
from tensorflow.keras import regularizers
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf
from sklearn.metrics import roc_auc_score

In [6]:
import datetime
str(datetime.date.today())[-5:]

'10-11'

In [4]:
#f.debugging.set_log_device_placement(True)

In [5]:
data_train =pd.read_csv('../01_inter_data/train_xgb_cleaned_01.csv')
data_test_a = pd.read_csv('../01_inter_data/test_a_xgb_cleaned_01.csv')

In [ ]:
from tensorflow.python.client import device_lib 


In [6]:
len(data_train.loc[data_train.isDefault == 1])

159610

In [7]:
features = [f for f in data_train.columns if f not in ['id','isDefault']]
x_train = data_train[features]
x_test = data_test_a[features]
y_train = data_train['isDefault']

In [8]:
scale = StandardScaler()
scale.fit(x_train)
x_train = scale.transform(x_train)

In [9]:
x_test = scale.transform(x_test)

In [10]:
# find class_weights
from sklearn.utils import class_weight

In [11]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)

In [12]:
class_weights ={i:class_weights[i] for i in range(2)}

In [13]:
class_weights

{0: 0.6246193725698402, 1: 2.506108639809536}

In [14]:
def auroc(y_true, y_pred):
    try:
        return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)
    except ValueError:
        pass
#    return score

In [15]:
nb_epoch = 600
batch_size = 128
feature_size = (x_train.shape[1])
fc_layer_units = (512,128,32,8,2)

In [16]:
def auc(y_true, y_pred):  
    ptas = tf.stack([binary_PTA(y_true,y_pred,k) for k in np.linspace(0, 1, 100)],axis=0)  
    pfas = tf.stack([binary_PFA(y_true,y_pred,k) for k in np.linspace(0, 1, 100)],axis=0)  
    pfas = tf.concat([tf.ones((1,)) ,pfas],axis=0)  
    binSizes = -(pfas[1:]-pfas[:-1])  
    s = ptas*binSizes  
    return K.sum(s, axis=0)  
#-----------------------------------------------------------------------------------------------------------------------------------------------------  
# PFA, prob false alert for binary classifier  
def binary_PFA(y_true, y_pred, threshold=K.variable(value=0.5)):  
    y_pred = K.cast(y_pred >= threshold, 'float32')  
    # N = total number of negative labels  
    N = K.sum(1 - y_true)  
    # FP = total number of false alerts, alerts from the negative class labels  
    FP = K.sum(y_pred - y_pred * y_true)  
    return FP/N  
#-----------------------------------------------------------------------------------------------------------------------------------------------------  
# P_TA prob true alerts for binary classifier  
def binary_PTA(y_true, y_pred, threshold=K.variable(value=0.5)):  
    y_pred = K.cast(y_pred >= threshold, 'float32')  
    # P = total number of positive labels  
    P = K.sum(y_true)  
    # TP = total number of correct alerts, alerts from the positive class labels  
    TP = K.sum(y_pred * y_true)  
    return TP/P  

In [17]:
def fc_block(X,units,dropout,stage):
    fc_name = 'fc' + str(stage)
    X = Dense(units,activation ='elu',name = fc_name,kernel_regularizer=regularizers.l1_l2(l1=5e-6, l2=5e-5))(X)
    X = Dropout(dropout)(X)
    
    return X

def nnModel():
    X_input = Input(shape=[feature_size])
    X = Dropout(0.0)(X_input)
    X = fc_block(X,fc_layer_units[0],dropout = 0.0,stage = 1)
    X = fc_block(X,fc_layer_units[1],dropout = 0.0,stage = 2)
    X = Dense(1,activation = 'sigmoid',name = 'fc3_output')(X)
    
    model = Model(inputs = X_input, outputs = X, name='nn_model')
    return model

In [18]:
def call_back():
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor = 0.2, patience=5, mode='auto', min_lr = 1e-7)
    early_stopping = EarlyStopping(verbose=1, patience=10, monitor='val_loss')  
    model_checkpoint = ModelCheckpoint(filepath='fengkongmlp.h5', verbose=1, save_best_only=True, monitor='val_loss')
    callbacks = [early_stopping, model_checkpoint]
    return callbacks

In [19]:
def train_trans_model():
    trans_model = load_model('fengkongmlp.h5',custom_objects={'auc':auc},compile = False)
    trans_model.compile(optimizer = Adam(lr=2e-5), loss = "binary_crossentropy", metrics = [auc])
    history_2 = trans_model.fit(x = x_train, y = y_train, validation_split=0.1, class_weight=class_weights,epochs = nb_epoch, batch_size = 64,callbacks=callbacks)
    return history_2

In [20]:
nn_model = nnModel()

In [21]:
nn_model.summary()

Model: "nn_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 147)]             0         
_________________________________________________________________
dropout (Dropout)            (None, 147)               0         
_________________________________________________________________
fc1 (Dense)                  (None, 512)               75776     
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
fc2 (Dense)                  (None, 128)               65664     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
fc3_output (Dense)           (None, 1)                 129

In [22]:
nn_model.compile(optimizer = Adam(lr= 2e-5), loss = "binary_crossentropy", metrics = [auc])

In [23]:
callbacks = call_back()

In [24]:
history = nn_model.fit(x = x_train, y = y_train, validation_split=0.1, class_weight=class_weights,epochs = nb_epoch, batch_size = batch_size,callbacks=callbacks)

Epoch 1/600
5617/5625 [============================>.] - ETA: 0s - loss: 0.6789 - auc: 0.7010
Epoch 00001: val_loss improved from inf to 0.67292, saving model to fengkongmlp.h5
5625/5625 [==============================] - 26s 5ms/step - loss: 0.6789 - auc: 0.7010 - val_loss: 0.6729 - val_auc: 0.7084
Epoch 2/600
5618/5625 [============================>.] - ETA: 0s - loss: 0.6703 - auc: 0.7097
Epoch 00002: val_loss did not improve from 0.67292
5625/5625 [==============================] - 26s 5ms/step - loss: 0.6703 - auc: 0.7097 - val_loss: 0.6741 - val_auc: 0.7101
Epoch 3/600
5619/5625 [============================>.] - ETA: 0s - loss: 0.6668 - auc: 0.7126
Epoch 00003: val_loss improved from 0.67292 to 0.67105, saving model to fengkongmlp.h5
5625/5625 [==============================] - 25s 5ms/step - loss: 0.6667 - auc: 0.7126 - val_loss: 0.6710 - val_auc: 0.7112
Epoch 4/600
5624/5625 [============================>.] - ETA: 0s - loss: 0.6640 - auc: 0.7147
Epoch 00004: val_loss improved 

Epoch 30/600
5624/5625 [============================>.] - ETA: 0s - loss: 0.6344 - auc: 0.7303
Epoch 00030: val_loss did not improve from 0.63397
5625/5625 [==============================] - 25s 4ms/step - loss: 0.6344 - auc: 0.7303 - val_loss: 0.6377 - val_auc: 0.7209
Epoch 31/600
5613/5625 [============================>.] - ETA: 0s - loss: 0.6338 - auc: 0.7302
Epoch 00031: val_loss did not improve from 0.63397
5625/5625 [==============================] - 25s 4ms/step - loss: 0.6338 - auc: 0.7302 - val_loss: 0.6436 - val_auc: 0.7220
Epoch 32/600
5617/5625 [============================>.] - ETA: 0s - loss: 0.6332 - auc: 0.7305
Epoch 00032: val_loss did not improve from 0.63397
5625/5625 [==============================] - 24s 4ms/step - loss: 0.6332 - auc: 0.7305 - val_loss: 0.6468 - val_auc: 0.7219
Epoch 33/600
5620/5625 [============================>.] - ETA: 0s - loss: 0.6327 - auc: 0.7311
Epoch 00033: val_loss did not improve from 0.63397
5625/5625 [==============================] -

In [ ]:
history.history.keys()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train', 'validate'], loc='upper left')

In [ ]:
plt.plot(history.history['auc'])
plt.plot(history.history['val_auc'])
plt.legend(['train_auc', 'validate_auc'], loc='upper left')

In [ ]:
plt.plot(np.append(history.history['loss'],history_2.history['loss']))
plt.plot(np.append(history.history['val_loss'],history_2.history['val_loss']))
plt.legend(['train', 'validate'], loc='upper left')

In [ ]:
plt.plot(np.append(history.history['auc'],history_2.history['auc']))
plt.plot(np.append(history.history['val_auc'],history_2.history['val_auc']))
plt.legend(['train', 'validate'], loc='upper left')